## Erap 1 - Uczenie Nadzorowane

In [1]:
from MODNet.train import supervised_training
import torch

Ścieżki do plików z danymi:

In [2]:
natural_path = "D:\\natural.h5"
synthetic_path = "D:\\synthetic.h5"
background_path = "D:\\backgrounds.h5"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training will use {device}")

#### Pęla uczenia

Checkpointy zapisywane są po obliczeniu metryk walidacyjnych.
- Stworzenie pliku ./training/supervised/STOP_TRAINING spowoduje przerwanie uczenia po następnym zapisie.
- Uczenie można wznowić podając opcję `resume=True`

Opcja `save_steps` określa jak często odbywa się walidacja i zapisywanie
- Większe wartości przyspieszają uczenie.
- Mniejsze wartości zmniejszają czas między zapisami, zwiększając prawdopodobieństwo zapisania najlepszego modelu i powodując częstsze czyszczenie pamięci GPU.

Większy `batch_size` powinno przyspieszyć uczenie, ale może spowodować, że na GPU skończy się pamięć.
- Przykładowe wartości: `save_steps=1000, batch_size=32`, `save_steps=2000, batch_size=16`, `save_steps=4000, batch_size=8`

In [ ]:
supervised_training(natural_path, synthetic_path, background_path, device, epochs=40, save_steps=4000, batch_size=48)

Po przerwaniu lub zakończeniu uczenia z jakiegokolwiek powodu **zrestartuj kernel Pythona**.

## Etap 2 - Uczenie Samonadzorowane

In [1]:
from MODNet.train import self_supervised_training
from MODNet.src.models.modnet import MODNet
import torch

Ścieżka do najlepszego modelu wytrenowanego w poprzednim etapie:

In [2]:
best_model_path = "./training/supervised/best.pth"

In [3]:
modnet = torch.nn.DataParallel(MODNet(backbone_pretrained=False))
modnet.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

Ścieżka do pliku z danymi:

In [4]:
file_path = "D:\\self-supervised.h5"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training will use {device}")

#### Pęla uczenia

Checkpointy zapisywane są po `save_steps` krokach.
- Stworzenie pliku ./training/supervised/STOP_TRAINING spowoduje przerwanie uczenia po następnym zapisie.
- Uczenie można wznowić podając opcję `resume=True`
- Mniejszy wpływ na uczenie niż w poprzednim etapie

In [ ]:
self_supervised_training(modnet, file_path, device, save_steps=10000)

Po przerwaniu lub zakończeniu uczenia z jakiegokolwiek powodu **zrestartuj kernel Pythona**.